In [1]:
!pip install datasets


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from datasets import Dataset

# QAペアのデータセットを作成
data = {
    "question": ["What is the capital of France?", "Who wrote 1984?", "What is the largest planet in our solar system?"],
    "answer": ["Paris", "George Orwell", "Jupiter"]
}

df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


/usr/local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def preprocess_function(examples):
    questions = examples["question"]
    answers = examples["answer"]
    inputs = tokenizer(questions, truncation=True, padding=True)
    inputs["labels"] = tokenizer(answers, truncation=True, padding=True)["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

trainer.train()


Map: 100%|██████████| 3/3 [00:00<00:00, 576.25 examples/s]


ValueError: The model did not return a loss from the inputs, only the following keys: start_logits,end_logits. For reference, the inputs it received are input_ids,attention_mask.

In [6]:
!pip install transformers datasets faiss-cpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 64.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [8]:
from datasets import load_dataset

# データセットのロード
dataset = load_dataset('wiki_dpr', 'psgs_w100')


ValueError: Loading wiki_dpr requires you to execute the dataset script in that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option `trust_remote_code=True` to remove this error.

In [9]:
from datasets import load_dataset

# データセットのロード
dataset = load_dataset('wiki_dpr', 'psgs_w100', trust_remote_code=True)


ValueError: BuilderConfig 'psgs_w100' not found. Available: ['psgs_w100.nq.exact', 'psgs_w100.nq.compressed', 'psgs_w100.nq.no_index', 'psgs_w100.multiset.exact', 'psgs_w100.multiset.compressed', 'psgs_w100.multiset.no_index', 'psgs_w100.nq.exact.no_embeddings', 'psgs_w100.nq.compressed.no_embeddings', 'psgs_w100.nq.no_index.no_embeddings', 'psgs_w100.multiset.exact.no_embeddings', 'psgs_w100.multiset.compressed.no_embeddings', 'psgs_w100.multiset.no_index.no_embeddings']

In [10]:
from datasets import load_dataset

# データセットのロード
dataset = load_dataset('wiki_dpr', 'psgs_w100.nq.exact', trust_remote_code=True)


KeyboardInterrupt: 

In [11]:
from datasets import load_dataset
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import faiss
import numpy as np

# データセットのロード
dataset = load_dataset('wiki_dpr', 'psgs_w100.nq.exact', trust_remote_code=True)

# モデルとトークナイザーのロード
model_name = "facebook/rag-sequence-nq"
tokenizer = RagTokenizer.from_pretrained(model_name)
retriever = RagRetriever.from_pretrained(model_name, use_dummy_dataset=True)
model = RagSequenceForGeneration.from_pretrained(model_name, retriever=retriever)

# ドキュメントのエンコーディング
def embed_passages(passages):
    embeddings = []
    for passage in passages:
        inputs = tokenizer(passage, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
        outputs = model.retriever.question_encoder(**inputs)
        embeddings.append(outputs.pooler_output.detach().numpy())
    return np.vstack(embeddings)

# ドキュメントのエンベッド
passages = dataset['train']['text'][:1000]  # デモのため、最初の1000ドキュメントのみを使用
passage_embeddings = embed_passages(passages)

# Faissインデックスの作成
index = faiss.IndexFlatL2(passage_embeddings.shape[1])
index.add(passage_embeddings)
faiss.write_index(index, "faiss_index")

# Faissインデックスをモデルに読み込む
retriever.index = faiss.read_index("faiss_index")

# 質問のトークナイズ
question = "What is the capital of France?"
inputs = tokenizer(question, return_tensors="pt")

# 回答の生成
outputs = model.generate(inputs["input_ids"])
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print(answer)
